In [1]:
# Step 3: Importing Syft
import syft as sy
sy.VERBOSE = False

# PART 1: Launch a Duet Server and Connect

As the data owner, we want to allow someone else to perform data science on data that we own ourselves. 

In order to do this, we must load our data into a locally running server (within our notebook). We call this server a "Duet". 

To begin, we must launch Duet and help our duet "partner" (a data scientist) connect to our server.

I do this by running the code below and following the instructions it gives!

In [3]:
duet = sy.launch_duet()

🎤  🎸  ♪♪♪ starting duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently 
♫♫♫ > in alpha. Do not use this to protect real-world data.
♫♫♫ >
♫♫♫ > Punching through firewall to OpenGrid Network Node at network_url: 
♫♫♫ > http://ec2-18-216-8-163.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... DONE!
♫♫♫ >
♫♫♫ > STEP 1: Send the following code to your duet partner!

import syft as sy
sy.VERBOSE=False
duet = sy.join_duet('1a22ce8679a0ccdd0146266718f363e7')

♫♫♫ > STEP 2: The code above will print out a 'Client Id'. Have
♫♫♫ >         your duet partner send it to you and enter it below!



♫♫♫ > Duet Partner's Client Id: 54c20a8c31f1588f6d37b7b9b08b5d0c


♫♫♫ > Connecting...
♫♫♫ > ...using a running event loop...

♫♫♫ > DUET LIVE STATUS  *  Objects: 1  Requests: 0   Messages: 6                                

# Part 2: Host My Data On My Duet Server

In order for me to allow a data scientist to perform data science on my information, I must first upload it to my Duet server (which I launched in the last cell)

I can send data to the server using the PySyft API. By default, when I send data to the server, only I can use it for computation.

If I send data with the "searchable=True" flag, then others who are connected to my Duet server will be able to find/use my data BUT NOT DOWNLOAD IT.

One more detail.... in order for people to find my data, I need to NAME IT so that they know which tensor on Duet is the one they are looking for

In [4]:
import torch as th

# let's pretend we have a tensor of human ages
age_data = th.tensor([54,25,32,53,16])

# Let's tag it as "age data"
age_data = age_data.tag("#age_data")

# Let's add a description to the tensor
age_data = age_data.describe("This is a list of people's ages. Let's keep it private!")

# Let's UPLOAD THE DATA to the Duet server!
age_data_pointer = age_data.send(duet, searchable=True)

In [9]:
# one way to see what's in our store is to just look at the variable
duet.store

In [5]:
# alternatively, we can pretty-print the data using a pandas table
duet.store.pandas

,ID,Tags,Description
0,<UID:8a99eeb6-0054-49f3-a917-14a44fc08b9e>,[#age_data],This is a list of people's ages. Let's keep it...


Exception in callback AsyncIOEventEmitter._emit_run.<locals>._callback(<Task finishe...ng instead.')>) at /Users/atrask/opt/anaconda3/envs/syft/lib/python3.7/site-packages/pyee/_asyncio.py:55
handle: <Handle AsyncIOEventEmitter._emit_run.<locals>._callback(<Task finishe...ng instead.')>) at /Users/atrask/opt/anaconda3/envs/syft/lib/python3.7/site-packages/pyee/_asyncio.py:55>
Traceback (most recent call last):
  File "/Users/atrask/opt/anaconda3/envs/syft/lib/python3.7/site-packages/nest_asyncio.py", line 190, in run
    ctx.run(self._callback, *self._args)
  File "/Users/atrask/opt/anaconda3/envs/syft/lib/python3.7/site-packages/pyee/_asyncio.py", line 62, in _callback
    self.emit('error', exc)
  File "/Users/atrask/opt/anaconda3/envs/syft/lib/python3.7/site-packages/pyee/_base.py", line 111, in emit
    self._emit_handle_potential_error(event, args[0] if args else None)
  File "/Users/atrask/opt/anaconda3/envs/syft/lib/python3.7/site-packages/pyee/_base.py", line 83, in _emit_handl

An <class 'RuntimeError'> has been triggered by <class 'syft.core.node.common.action.run_class_method_action.RunClassMethodAction'> 

An AuthorizationException has been triggered by <class 'syft.core.node.common.action.get_object_action.GetObjectAction'> from <Address - Domain:<SpecificLocation:..c2356>>


# Part 3: Wait for Data Scientist to do Analysis

During this section, we wait as the data scientist uses our data to answer a question. The key constraint is that while the data scientist can perform analysis, they have to ask permission to download any results for them to see.

So while they're doing analysis, we will continue to CHECK to see if they have REQUESTED permission to download anything using the following code

In [6]:
# run this repeatedly to check if there are requests
duet.requests.pandas

,Request Name,Reason,Request ID,Requested Object's ID
0,Age Request,I am a data scientist and I want to know the a...,<UID:4c9c4ff9-5a02-438c-a504-b67c71a9a095>,<UID:ba56d6a0-e4e7-414a-902f-ee67dc3b6b7c>


In [7]:
# Looks like we have a request for data! Let's take a look!

duet.requests[0].request_description

'I am a data scientist and I want to know the average age of the people in your dataset.'

In [8]:
# this looks like a great request... 
# I could deny it by calling .deny(), but I'm going to accept it

#duet.requests[0].deny()

duet.requests[0].accept()

Granting request: <UID:4c9c4ff9-5a02-438c-a504-b67c71a9a095>


In [9]:
# run this repeatedly to check if there are requests
duet.requests.pandas

""
